In [2]:
import copy
import gc
import calendar

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.externals import joblib
from xgboost import XGBRegressor

print('packages loaded')

packages loaded




Load saved cleaned df 

In [28]:
#evaluate the cell below before this one

XY_val_fp = 'final_data/train_val.csv'
XY_test_fp = 'final_data/train_test.csv'


train_save_model(XY_val_fp, XY_test_fp)

Nulls_filled:
wtd_temp_mean          47434
wtd_temp               46763
kelv                   46763
kelv2                  46763
diff_from_av_T         46763
diff_from_av_T2        46763
f1d4_Values                4
bd1_Values                 8
f1d1_Values                2
shift_1Df1d4_Values      100
shift_1Df1d1_Values       98
shift_1Dbd1_Values        96
shift_7Df1d4_Values      676
shift_7Df1d1_Values      674
shift_7Dbd1_Values       672
p1shft                    96
peak1shft                 95
t1shft                 46859
p2shft                   192
peak2shft                191
t2shft                 46955
p3shft                   288
peak3shft                287
t3shft                 47051
p7shft                   672
peak7shft                671
t7shft                 47435
p364shft               34944
peak364shft            34943
t364shft               81707
p1shft_d               46763
p7sfht_d               46763
diff_from_av_T_sq      46763
temp_streak_av         46763


In [27]:

def load_feats(XY_fp):
    """load features and format time columns"""
    XY = pd.read_csv('final_data/train_val.csv')
    XY['Timestamp'] = pd.to_datetime(XY.Timestamp, format='%Y-%m-%d %H:%M:%S')
    XY['date'] = XY['Timestamp'].dt.date
    return XY

def train_save_model(XY_val_fp, XY_test_fp, forecast='15min', model_type='xgboost', compute_cores=4):
    """Save trained model and validation results for specified model and forecast period.
    
    XY_val_fp - file path to the dataframe of the feats+target for validation
    XY_test_fp - file path to the dataframe of the feats+target for final testing
    forcast - Select one from ['15min', '1hour', '1day']. Specifies how far out the forecast is.
    model_type - Select one from ['xgboost', 'lstm']. Which model to use."""

    
    # define sets of columns that aren't valid features
    not_1hour_ahead_feats = set(['ahead_15', 'ahead_15_diff' ,'ahead_30'])

    not_day_ahead_feats = set(['ahead_15', 'ahead_15_diff' ,'ahead_30',
                                 '1hr_ahead', '1hr_ahead_diff', '1p5hr_ahead', '2hr_ahead'])
    
    always_not_feats =  set(['arb_id','meter_id','obs_id', 'day','tdelt', 'DofW','diff_from_av','pred_values',
                             'date','Timestamp','targ','863','938','relativemae','diff_rat','peak_hour',
                             'dist_from_peak', 'peak_hour','data_set','f1d4_Values','bd1_Values',  'f1d1_Values',
                         'obs_id','date','is_abnormal','Values','234_203','diff1','tdiff1'])

    if forecast =='15min':
        not_feats = always_not_feats
    elif forecast =='1hour':
        not_feats = always_not_feats.union(not_1hour_ahead_feats)
    elif forecast =='1day':
        not_feats = always_not_feats.union(not_day_ahead_feats)
    else:
        raise ValueError

    # x_colsd = joblib.load( 'day_ahead_feats.pkl') 
    if model_type == 'xgboost':
        
        #tune model parameters
        #to be added
        
        #final validation results
        save_tuned_model_results(XY_val_fp, forecast, model_type, compute_cores, not_feats, val_not_test=True)
        
        #final test
        save_tuned_model_results(XY_test_fp, forecast, model_type, compute_cores, not_feats, val_not_test=False)
        
def get_features(df, not_feats):
    """Return list of features for model"""
    x_feat_names = list(set(df) - not_feats)
    y_feat_name ='diff_from_av'
    
    assert not y_feat_name in x_feat_names
    
    return x_feat_names, y_col

        
def save_tuned_model_results(XY_fp, forecast, model_type, compute_cores, not_feats,  val_not_test=True):
    XY = pd.read_csv(XY_fp)
    x_cols, y_col = get_features(XY, not_feats)
    
    original_null_values = XY.isnull().sum()
    original_null_values=original_null_values[original_null_values>0]
    XY.fillna(-300, inplace=True)
    print('Nulls_filled:')
    print(original_null_values)
     
    #get mask for train data
    train_mask = (XY['data_set']==0)
    not_train_mask = ~train_mask
    
    X_train, X_test = XY.loc[train_mask, x_cols].copy(), XY.loc[not_train_mask,x_cols].copy()
    y_train, y_test = XY.loc[train_mask, y_col].copy(), XY.loc[not_train_mask, y_col].copy()

    
    tuned_parameter_filepath = 'modelparams_' + forecast + '_' + model_type + '.pkl'
    model_params = joblib.load(tuned_parameter_filepath)
    
    reg = XGBRegressor(max_depth=model_params['max_depth'],
                   learning_rate=model_params['learning_rate'],
                   colsample_bylevel =model_params['colsample_bylevel'],
                   min_child_weight=model_params['min_child_weight'],
                   n_estimators=model_params['best_iteration'],
                   colsample_bytree=model_params['colsample_bytree'],
                   reg_alpha=model_params['reg_alpha'], 
                   reg_lambda=model_params['reg_lambda'],
                   n_jobs=compute_cores)
    
    if val_not_test:
        eval_set = [[X_test, y_test]]
        reg.fit(X_train, y_train,
               eval_set=eval_set,
               early_stopping_rounds=50, verbose=True )
        
        
        print(reg.best_iteration)
        
    else:
        reg.fit(X_train, y_train)
    
    XY['pred_change'] = reg.predict(XY[x_cols])
    XY['err'] =  XY[y_col] - XY['pred_change']

    # the prediciton was the dif from av
    XY['pred'] = XY['pred_change'] + XY['hour_av']
    XY['err'] = XY['pred_change'] - XY['diff_from_av']
    XY.index = XY.Timestamp

    XY['AE'] = XY['err'].apply(abs)
    XY['PE'] = XY['err'].divide(XY['Values'])
    XY['APE'] = XY['PE'].apply(abs)

    XY['1p2hourerr'] = XY['err'].rolling(2, center=True).sum()
    XY['1hourerr'] = XY['err'].rolling(4, center=True).sum()
    XY['4hourerr'] = XY['err'].rolling(4*4, center=True).sum()
    XY['1week'] = XY['err'].rolling(7*4*24, center=True).sum()
    XY['1day'] = XY['err'].rolling(1*4*24, center=True).sum()

    print('MAPE: ', XY.loc[XY.data_set!=0,'APE'].mean())
    print('MAE: ', XY.loc[XY.data_set!=0,'AE'].mean())
    
    #save results and model
    if val_not_test:
        val_test = 'val'
    else:
        val_test = 'test'
    
    augmented_df_filepath =  ('model_result' + forecast + '_' +
                              model_type + '_' + val_test + '.csv')
    XY.to_csv(augmented_df_filepath, index=False) 
        #storage is not a problem but should shave only new cols
     
    feature_list_file_path =  ('model_feat_list' + forecast + '_' + 
                               model_type + '_' + val_test + '.pkl')
    joblib.dump(x_cols, feature_list_file_path)
    
    train_model_filepath =  ('model_trained' + forecast + '_' +
                             model_type + '_' + val_test + '.pkl')
    joblib.dump(reg, train_model_filepath) 
    
    del XY
    del X_train
    del X_test
    del y_train
    del y_test